In [ ]:
#%kaggle datasets download -d warcoder/gkn-blade-surface-defect-dataset

In [ ]:
#import zipfile

# Substitua o nome do arquivo pelo nome correto
#with zipfile.ZipFile("gkn-blade-surface-defect-dataset.zip", 'r') as zip_ref:
#    zip_ref.extractall("dados_extraidos")  # Pasta onde os dados serão extraídos


# Classificação de imagens

In [ ]:
#%ls

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,random_split

# Transformações para pré-processamento

# Dividir em treino e teste

# Calcular média e desvio padrão apenas no treino

# Aplicar transformações finais com normalização calculada


# Criar DataLoaders

# Criando um modelo do zero

Como você estamos aplicando `transforms.Resize((224, 224))`, é necessário verificar se o tamanho da entrada da camada totalmente conectada (`fc1`) está correto. Vamos calcular isso.

---

### 📌 **Passo 1: Como as Camadas Convolucionais e Pooling Afetam o Tamanho?**
Cada convolução (`Conv2d`) com `padding=1` e `kernel_size=3` mantém as dimensões, enquanto cada camada de `MaxPool2d(2,2)` reduz pela metade.

#### **Transformação passo a passo para uma entrada de (224, 224):**
1. **Entrada inicial**:  $ 224 \times 224  $
2. **Após `conv1` (Conv2d(3, 32, 3, padding=1))** → Dimensão **continua** $$  224 \times 224  $$
3. **Após `pool` (MaxPool2d(2,2))** → $  224 / 2 = 112 \times 112  $ 
4. **Após `conv2` (Conv2d(32, 64, 3, padding=1))** → $  112 \times 112  $ *(não muda devido ao padding)*
5. **Após `pool` (MaxPool2d(2,2))** → $  112 / 2 = 56 \times 56  $
6. **Após `conv3` (Conv2d(64, 128, 3, padding=1))** → $ 56 \times 56  $*(não muda devido ao padding)*
7. **Após `pool` (MaxPool2d(2,2))** → $  56 / 2 = 28 \times 28  $

---

### 📌 **Passo 2: Ajustando o `fc1`**
O número de neurônios que entram na camada `fc1` é:


$ 128 \times 28 \times 28 $


Logo, **a linha correta do `fc1` deve ser**:
```python
self.fc1 = nn.Linear(128 * 28 * 28, 512)
```



# Criar o modelo

# Definir a Função de Perda e o Otimizador

# Treinar o modelo

# Avaliar no conjunto de teste

# Outro caminho - Transferência de aprendizagem

https://pytorch.org/vision/main/models.html

Esse código está usando **Transfer Learning** com o modelo **ResNet-18** pré-treinado no ImageNet. Vamos entender linha por linha:

---

### 🔹 **Linha 1: Definição do Número de Classes**
```python
num_classes = 3
```
Aqui, definimos que o dataset possui **3 classes** (exemplo: `['Good', 'Nick', 'Scratch']`). O modelo precisa ajustar a saída para esse número.

---

### 🔹 **Linha 2: Carregamento da ResNet-18 Pré-Treinada**
```python
model = models.resnet18(pretrained=True)
```
- Baixa a **ResNet-18** pré-treinada no **ImageNet** (que tem 1.000 classes).
- O modelo já aprendeu a detectar padrões básicos como bordas, texturas e formas, o que pode ser reutilizado para novas tarefas.

---

### 🔹 **Linha 3: Obtendo o Número de Entradas da Última Camada**
```python
num_ftrs = model.fc.in_features
```
- O modelo ResNet-18 tem uma **camada totalmente conectada (fc)** na saída.
- Essa camada original tem **1.000 neurônios** (porque o ImageNet tem 1.000 classes).
- `model.fc.in_features` retorna **o número de entradas dessa camada** (ou seja, quantos neurônios da penúltima camada conectam na saída).
- Na ResNet-18, esse valor geralmente é **512**.

---

### 🔹 **Linha 4: Substituindo a Última Camada**
```python
model.fc = nn.Linear(num_ftrs, num_classes)
```
- Como nosso dataset tem apenas **3 classes**, trocamos a última camada.
- A nova `fc` agora recebe **512 entradas** (da penúltima camada da ResNet) e **3 saídas** (uma para cada classe).

---

### 🔹 **Resumo**
1. **Baixamos a ResNet-18 pré-treinada** para reutilizar suas camadas convolucionais.
2. **Descobrimos quantas features são passadas para a última camada (`fc`)**.
3. **Trocamos a última camada** para que a rede aprenda a classificar apenas 3 classes, ao invés das 1.000 do ImageNet.

---

### 🔹 **Fluxo do Modelo Antes e Depois**
**Antes (original da ResNet-18 no ImageNet):**
```
Camadas convolucionais → Global Avg Pooling → nn.Linear(512, 1000)
```
**Depois (ajustado para nosso problema):**
```
Camadas convolucionais → Global Avg Pooling → nn.Linear(512, 3)
```
Ou seja, **mantemos todo o aprendizado da ResNet-18**, mas adaptamos a saída para nosso próprio problema! 🚀🔥

# Treinar o modelo

# Avaliar no Conjunto de teste

# Salvar e carregar o modelo

# Fazer previsões em novas imagens